In [1]:
import numpy  as np
import pandas as pd
from metrics import MSE
from tqdm import tqdm

In [2]:
X_train = pd.read_pickle("../Datasets/final/X_train.pkl")
c = X_train.columns.tolist()
X_val = pd.read_pickle("../Datasets/final/X_val.pkl")
X_test = pd.read_pickle("../Datasets/final/X_test.pkl")
y_train = pd.read_pickle("../Datasets/final/y_train.pkl").to_numpy()
y_val = pd.read_pickle("../Datasets/final/y_val.pkl").to_numpy()
y_test = pd.read_pickle("../Datasets/final/y_test.pkl").to_numpy()

In [3]:
X_train.insert(X_train.shape[1], "rating", y_train.T[0])

In [4]:
category_columns = [i for i in c if "category" in i]

In [5]:
mean_per_category_dict = {}
for category in category_columns:
    category_df = X_train.groupby([category, "gPlusPlaceId"]).agg({"gPlusUserId": "count"}).reset_index()
    popular_place = category_df[category_df[category] == 1].sort_values("gPlusUserId", ascending=False).iloc[0]["gPlusPlaceId"]
    mean_per_category_dict[category] = X_train[(X_train[category] == 1) & (X_train["gPlusPlaceId"] == popular_place)]["rating"].mean()
    

In [6]:
mean_per_category_dict

{'final_category_AssocOrgs': 5.0,
 'final_category_Entertainment': 4.064516129032258,
 'final_category_Legal': 2.3333333333333335,
 'final_category_Medical': 1.9,
 'final_category_Others': 4.776859504132231,
 'final_category_Public': 4.962962962962963,
 'final_category_Restaurant': 3.868131868131868,
 'final_category_School': 4.372549019607843,
 'final_category_ShopsStores': 4.346666666666667,
 'final_category_Venues': 3.933333333333333}

In [7]:
y_pred_val = []
for i in tqdm(range(len(X_val))):
    for j in range(len(X_val.iloc[i][category_columns])):
        if X_val.iloc[i][category_columns][j] == 1:
            y_pred_val.append(mean_per_category_dict[category_columns[j]])
            break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 161526/161526 [03:45<00:00, 716.48it/s]


In [8]:
MSE(predictions=y_pred_val, labels=y_val)[0]

1.8246654801945392

In [9]:
y_pred = []
for i in tqdm(range(len(X_test))):
    for j in range(len(X_test.iloc[i][category_columns])):
        if X_test.iloc[i][category_columns][j] == 1:
            y_pred.append(mean_per_category_dict[category_columns[j]])
            break

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 161526/161526 [03:10<00:00, 849.78it/s]


In [10]:
MSE(predictions=y_pred, labels=y_test)[0]

1.801757653179724